# 2. Составление списка генов для извлечения полных белковых последовательностей для БД VDJdb

Антон Смирнов, 13.11.2022

In [51]:
import pandas as pd
import json
from datetime import datetime

In [30]:
vdjdb = pd.read_csv("../data/VDJdb-2022-11-13 22_41.tsv", sep = "\t", header=0)
vdjdb.head()

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2


In [31]:
vdjdb[["Score","MHC class"]].value_counts()

Score  MHC class
0      MHCI         75587
1      MHCI          7337
0      MHCII         2091
2      MHCI          1690
3      MHCI          1546
1      MHCII          516
2      MHCII          381
3      MHCII          173
dtype: int64

In [32]:
vdjdb["Species"].value_counts()

HomoSapiens      81879
MusMusculus       5323
MacacaMulatta     2119
Name: Species, dtype: int64

Сайты разрезания ищем только для человеческих МНС 1 класса

In [33]:
def isCorrectSequence(seq):
    alphabet = list("ACDEFGHIKLMNPQRSTVWY")
    seq = seq.strip()
    for i in range(0,len(seq)):
        if seq[i] not in alphabet:
            return False
    return True

In [34]:
vdjdb["correct_epitope"] = vdjdb["Epitope"].apply(isCorrectSequence)
vdjdb.shape

(89321, 18)

In [35]:
vdjdb_filtered = vdjdb[vdjdb["correct_epitope"]]
vdjdb_filtered.shape

(89321, 18)

In [36]:
vdjdb_filtered = vdjdb_filtered[vdjdb_filtered["MHC class"] == "MHCI"]
vdjdb_filtered.shape

(86160, 18)

In [37]:
vdjdb_filtered = vdjdb_filtered[~vdjdb_filtered["Epitope gene"].isna()]
vdjdb_filtered.shape

(86126, 18)

In [38]:
vdjdb_filtered = vdjdb_filtered[~vdjdb_filtered["Epitope species"].isna()]
vdjdb_filtered.shape

(86126, 18)

In [39]:
vdjdb_filtered = vdjdb_filtered[~vdjdb_filtered["Reference"].isna()]
vdjdb_filtered.shape

(84615, 18)

In [40]:
vdjdb_filtered = vdjdb_filtered[vdjdb_filtered["Species"] == "HomoSapiens"]
vdjdb_filtered.shape

(77227, 18)

In [41]:
vdjdb_filtered["Epitope species"].value_counts()

CMV                          37945
EBV                          11291
InfluenzaA                   10509
SARS-CoV-2                    7135
HomoSapiens                   4868
HIV-1                         2687
HCV                            848
YFV                            789
HTLV-1                         234
DENV1                          182
DENV3/4                        179
HIV                            157
HSV-2                          154
synthetic                       68
DENV2                           60
HCoV-HKU1                       57
HPV                             28
MCPyV                           12
M.tuberculosis                   8
StreptomycesKanamyceticus        6
HIV1                             4
Homo sapiens                     4
SaccharomycesCerevisiae          2
Name: Epitope species, dtype: int64

In [42]:
vdjdb_filtered["Epitope gene"].value_counts()

IE1        28142
M          10038
pp65        8941
EBNA4       5032
Spike       2896
           ...  
TUBGCP2        1
RNF213         1
SNX24          1
ZFYVE27        1
PHLPP1         1
Name: Epitope gene, Length: 177, dtype: int64

In [47]:
vdjdb_filtered = vdjdb_filtered[(vdjdb_filtered["Epitope species"] != "synthetic") & (vdjdb_filtered["Epitope gene"] != "synthetic")]
vdjdb_filtered.shape

(77157, 18)

In [48]:
gene_species = dict()
for i in vdjdb_filtered.index:
    key = vdjdb_filtered.loc[i,"Epitope gene"]
    if key not in gene_species.keys():
        gene_species[key] = vdjdb_filtered.loc[i,"Epitope species"]
print(gene_species)

{'Nef': 'HIV-1', 'pp65': 'CMV', 'Nucleocapsid': 'SARS-CoV-2', 'ORF3': 'SARS-CoV-2', 'ORF1ab': 'SARS-CoV-2', 'Spike': 'SARS-CoV-2', 'NDC1': 'HomoSapiens', 'TKT': 'HomoSapiens', 'SEC24A': 'HomoSapiens', 'AKAP13': 'HomoSapiens', 'EXOC8': 'HomoSapiens', 'PABPC1': 'HomoSapiens', 'MLANA': 'HomoSapiens', 'BRLF1': 'EBV', 'Gag': 'HIV-1', 'IE1': 'CMV', 'EBNA1': 'EBV', 'BZLF1': 'EBV', 'Tax': 'HTLV-1', 'EBNA3A': 'EBV', 'M': 'InfluenzaA', 'NY-ESO-1': 'HomoSapiens', 'UL40': 'CMV', 'EBNA6': 'EBV', 'Tel1': 'SaccharomycesCerevisiae', 'ABCD3': 'HomoSapiens', 'BMLF1': 'EBV', 'ELAVL4': 'HomoSapiens', 'INS': 'HomoSapiens', 'Pol': 'HIV-1', 'TERT': 'HomoSapiens', 'NS3': 'HCV', 'EMC': 'HomoSapiens', 'WT1': 'HomoSapiens', 'MAGE-A3': 'HomoSapiens', 'TITIN': 'HomoSapiens', 'GAG': 'HIV-1', 'Leader peptide': 'HomoSapiens', 'RT': 'HIV-1', 'NP338': 'InfluenzaA', 'ANKRD30A': 'HomoSapiens', 'MAGE-A4': 'HomoSapiens', 'PMEL': 'HomoSapiens', 'P53': 'HomoSapiens', 'ENR': 'M.tuberculosis', 'TP53': 'Homo sapiens', 'PIK3CA':

In [53]:
with open("../lists/vdjdb_gene_species_" + datetime.today().strftime('%Y-%m-%d %H:%M:%S') + ".json", 'w') as f:
    json.dump(gene_species, f, sort_keys=True, indent=4)

 Подправлены вручную названия видов, удалены записи с сомнительными записями генов и видов. Файл 22-111-14 00:25